In [1]:
# Install dependencies
!pip install flask pyngrok diffusers transformers accelerate torch torchvision --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.4 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# --- Imports ---
import torch
from flask import Flask, request, jsonify, send_file
from pyngrok import ngrok
import shutil
import zipfile
import os
from PIL import Image
from diffusers import StableDiffusionPipeline
from transformers import BlipProcessor, BlipForConditionalGeneration
from google.colab import drive # Add this import


In [ ]:
# --- Config ---
NGROK_AUTH_TOKEN = "XXXXX"  # 🔐 Replace with your actual token

In [5]:
# Authenticate ngrok
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [8]:

# --- Initialize Models ---

# Stable Diffusion (image generation)
device = "cuda" if torch.cuda.is_available() else "cpu"
"""sd_pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16 if device=="cuda" else torch.float32
).to(device)"""

"""sd_pipe = StableDiffusionPipeline.from_pretrained(
    "hakurei/waifu-diffusion", torch_dtype=torch.float16 if device=="cuda" else torch.float32
).to(device)"""

"""sd_pipe = StableDiffusionPipeline.from_pretrained(
    "nitrosocke/Arcane-Diffusion", torch_dtype=torch.float16 if device=="cuda" else torch.float32
).to(device)"""

sd_pipe = StableDiffusionPipeline.from_pretrained(
    "Lykon/dreamshaper-8", torch_dtype=torch.float16 if device=="cuda" else torch.float32
).to(device)

# BLIP (caption generation)
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

# --- Helper Functions ---

def generate_images(prompt, num_images=5):
    image_paths = []
    for i in range(num_images):
        image = sd_pipe(prompt).images[0]
        img_path = f"img_{i}.png"
        image.save(img_path)
        image_paths.append(img_path)
    return image_paths

def generate_captions(image_paths):
    captions = []
    for path in image_paths:
        raw_image = Image.open(path).convert("RGB")
        inputs = blip_processor(raw_image, return_tensors="pt").to(device)
        out = blip_model.generate(**inputs)
        caption = blip_processor.decode(out[0], skip_special_tokens=True)
        captions.append(caption)
    return captions


# --- Flask App ---

app = Flask(__name__)
public_url = ngrok.connect(5000)
print("🚀 Backend running at:", public_url)

@app.route("/generate", methods=["POST"])
def generate():
    prompt = request.json.get("prompt")

    # Clean up old outputs
    if os.path.exists("output"): shutil.rmtree("output")
    os.makedirs("output", exist_ok=True)

    # 1. Generate images
    image_paths = generate_images(prompt)

    # 2. Generate captions
    captions = generate_captions(image_paths)

    # 3. Save to output dir
    for i, img_path in enumerate(image_paths):
        shutil.move(img_path, f"output/img_{i}.png")
        with open(f"output/img_{i}.txt", "w") as f:
            f.write(captions[i])

    # 4. Zip the results
    zipf = zipfile.ZipFile("output.zip", 'w')
    for file in os.listdir("output"):
        zipf.write(os.path.join("output", file), arcname=file)
    zipf.close()

    # 5. Copy output.zip to Google Drive
    google_drive_path = "/content/drive/MyDrive/GeneratedImages/" # You can change this path
    os.makedirs(google_drive_path, exist_ok=True) # Create the folder if it doesn't exist

    # Define a unique filename for the zip file in Google Drive to avoid overwriting
    # You might want to add a timestamp or part of the prompt to the filename
    #import datetime
    #timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    #drive_zip_filename = f"output_{timestamp}.zip"
    shutil.copy("output.zip", os.path.join(google_drive_path, "output.zip"))
    print(f"✅ output.zip saved to Google Drive at: {os.path.join(google_drive_path, 'output.zip')}")


    return jsonify({"download_url": f"{public_url}/download", "drive_status": "File saved to Google Drive"})

@app.route("/download", methods=["GET"])
def download():
    return send_file("output.zip", as_attachment=True)

# Run the Flask app
app.run(port=5000)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

🚀 Backend running at: NgrokTunnel: "https://ef4b-35-247-24-46.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [23/May/2025 11:32:28] "POST /generate HTTP/1.1" 200 -


✅ output.zip saved to Google Drive at: /content/drive/MyDrive/GeneratedImages/output.zip
